# Lesson 4: Constructing a Knowledge Graph from Text Documents

### Import packages and set up Neo4j

In [1]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI


# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# Note the code below is unique to this course environment, and not a 
# standard part of Neo4j's integration with OpenAI. Remove if running 
# in your own environment.
OPENAI_ENDPOINT = os.getenv('OPENAI_BASE_URL') + '/embeddings'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

### Take a look at a Form 10-K json file

- Publicly traded companies are required to fill a form 10-K each year with the Securities and Exchange Commision (SEC)
- You can search these filings using the SEC's [EDGAR database](https://www.sec.gov/edgar/search/)
- For the next few lessons, you'll work with a single 10-K form for a company called [NetApp](https://www.netapp.com/)

In [40]:
first_file_name = "./data/form10k/aapl-20210925.json"


In [41]:
first_file_as_object = json.load(open(first_file_name))
sections = first_file_as_object['sections']
print(sections)

[{'secDocumentUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm', 'itemsCnt': 4, 'items': [{'item': 'Item 1.', 'itemName': 'Business', 'itemValue': {'Markets and Distribution': 'The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.', 'Item 1.\xa0\xa0\xa0\xa0Business': '', 'Available Information': 'The Company’

In [42]:
items = [section['items'] for section in sections]
print(items)
type(items)


[[{'item': 'Item 1.', 'itemName': 'Business', 'itemValue': {'Markets and Distribution': 'The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.', 'Item 1.\xa0\xa0\xa0\xa0Business': '', 'Available Information': 'The Company’s Annual Reports on Form 10-K, Quarterly Reports on Form 10-Q, Current Reports on Form 8-K, and amendments to reports filed pursua

list

In [48]:
def clean_item_value(value):
    return value.lower().replace(' ', '').replace('.', '')



# Extract key-value pairs into a dictionary
extracted_list = []

for sublist in items:
    for dictionary in sublist:
        cleaned_dict = {k: clean_item_value(v) if k == 'item' else v for k, v in dictionary.items()}
        extracted_list.append(cleaned_dict)
print(extracted_list)



[{'item': 'item1', 'itemName': 'Business', 'itemValue': {'Markets and Distribution': 'The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.', 'Item 1.\xa0\xa0\xa0\xa0Business': '', 'Available Information': 'The Company’s Annual Reports on Form 10-K, Quarterly Reports on Form 10-Q, Current Reports on Form 8-K, and amendments to reports filed pursuant 

In [51]:
data = {}

# Extract the desired structure
for item_data in extracted_list:
    item_key = item_data['item']  # Extract the key for 'item'
    item_value = item_data['itemValue']  # Extract the value for 'itemValue'
    
    # Update the 'data' dictionary
    data[item_key] = item_value


print(data.keys())
print(data)

dict_keys(['item1', 'item1a', 'item7', 'item7a'])
{'item1': {'Markets and Distribution': 'The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.', 'Item 1.\xa0\xa0\xa0\xa0Business': '', 'Available Information': 'The Company’s Annual Reports on Form 10-K, Quarterly Reports on Form 10-Q, Current Reports on Form 8-K, and amendments to reports filed pursu

In [63]:
item1_text = data['item1'].values()
print(item1_text)


# Extract the string from dict_values object
flattened_item1_text = next(iter(item1_text)) 

# Remove leading and trailing quotes (if any)
if flattened_item1_text.startswith("'") and flattened_item1_text.endswith("'"):
    flattened_string = flattened_string[1:-1]
elif flattened_item1_text.startswith('"') and flattened_item1_text.endswith('"'):
    flattened_string = flattened_item1_text[1:-1]

print(flattened_item1_text)

dict_values(['The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.', '', 'The Company’s Annual Reports on Form 10-K, Quarterly Reports on Form 10-Q, Current Reports on Form 8-K, and amendments to reports filed pursuant to Sections 13(a) and 15(d) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), are filed with the U.S. Securit

### Split Form 10-K sections into chunks
- Set up text splitter using LangChain
- For now, split only the text from the "item 1" section 

In [64]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

In [65]:
item1_text_chunks = text_splitter.split_text(flattened_item1_text)

In [66]:
type(item1_text_chunks)

list

In [67]:
len(item1_text_chunks)

1

In [68]:
item1_text_chunks[0]

'The Company’s customers are primarily in the consumer, small and mid-sized business, education, enterprise and government markets. The Company sells its products and resells third-party products in most of its major markets directly to consumers, small and mid-sized businesses, and education, enterprise and government customers through its retail and online stores and its direct sales force. The Company also employs a variety of indirect distribution channels, such as third-party cellular network carriers, wholesalers, retailers and resellers. During 2021, the Company’s net sales through its direct and indirect distribution channels accounted for 36% and 64%, respectively, of total net sales.'

- Set up helper function to chunk all sections of the Form 10-K
- You'll limit the number of chunks in each section to 20 to speed things up

In [71]:
def split_form10k_data_from_file(file):
    chunks_with_metadata = [] # use this to accumlate chunk records
    file_as_object = json.load(open(file)) # open the json file
    for item in ['item1','item1a','item7','item7a']: # pull these keys from the json
        print(f'Processing {item} from {file}') 
        item_text = file_as_object[item] # grab the text of the item
        item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
        chunk_seq_id = 0
        for chunk in item_text_chunks[:20]: # only take the first 20 chunks
            form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
            # finally, construct a record with metadata and the chunk text
            chunks_with_metadata.append({
                'text': chunk, 
                # metadata from looping...
                'f10kItem': item,
                'chunkSeqId': chunk_seq_id,
                # constructed metadata...
                'formId': f'{form_id}', # pulled from the filename
                'chunkId': f'{form_id}-{item}-chunk{chunk_seq_id:04d}',
                # metadata from file...
                'names': file_as_object['names'],
                'cik': file_as_object['cik'],
                'cusip6': file_as_object['cusip6'],
                'source': file_as_object['source'],
            })
            chunk_seq_id += 1
        print(f'\tSplit into {chunk_seq_id} chunks')
    return chunks_with_metadata

In [72]:
first_file_chunks = split_form10k_data_from_file(first_file_name)

Processing item1 from ./data/form10k/aapl-20210925.json


KeyError: 'item1'

In [ ]:
first_file_chunks[0]

### Create graph nodes using text chunks

In [ ]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.names = $chunkParam.names,
        mergedChunk.formId = $chunkParam.formId, 
        mergedChunk.cik = $chunkParam.cik, 
        mergedChunk.cusip6 = $chunkParam.cusip6, 
        mergedChunk.source = $chunkParam.source, 
        mergedChunk.f10kItem = $chunkParam.f10kItem, 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
        mergedChunk.text = $chunkParam.text
RETURN mergedChunk
"""

- Set up connection to graph instance using LangChain

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

- Create a single chunk node for now

In [ ]:
kg.query(merge_chunk_node_query, 
         params={'chunkParam':first_file_chunks[0]})

- Create a uniqueness constraint to avoid duplicate chunks

In [ ]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")


In [ ]:
kg.query("SHOW INDEXES")

- Loop through and create nodes for all chunks
- Should create 23 nodes because you set a limit of 20 chunks in the text splitting function above

In [ ]:
node_count = 0
for chunk in first_file_chunks:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunkId']}")
    kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

In [ ]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

### Create a vector index

In [ ]:
kg.query("""
         CREATE VECTOR INDEX `form_10k_chunks` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

In [ ]:
kg.query("SHOW INDEXES")

### Calculate embedding vectors for chunks and populate index
- This query calculates the embedding vector and stores it as a property called `textEmbedding` on each `Chunk` node.

In [ ]:
kg.query("""
    MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
    WITH chunk, genai.vector.encode(
      chunk.text, 
      "OpenAI", 
      {
        token: $openAiApiKey, 
        endpoint: $openAiEndpoint
      }) AS vector
    CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
    """, 
    params={"openAiApiKey":OPENAI_API_KEY, "openAiEndpoint": OPENAI_ENDPOINT} )

In [ ]:
kg.refresh_schema()
print(kg.schema)

### Use similarity search to find relevant chunks

- Setup a help function to perform similarity search using the vector index

In [ ]:
def neo4j_vector_search(question):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    WITH genai.vector.encode(
      $question, 
      "OpenAI", 
      {
        token: $openAiApiKey,
        endpoint: $openAiEndpoint
      }) AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
    RETURN score, node.text AS text
  """
  similar = kg.query(vector_search_query, 
                     params={
                      'question': question, 
                      'openAiApiKey':OPENAI_API_KEY,
                      'openAiEndpoint': OPENAI_ENDPOINT,
                      'index_name':VECTOR_INDEX_NAME, 
                      'top_k': 10})
  return similar

- Ask a question!

In [ ]:
search_results = neo4j_vector_search(
    'In a single sentence, tell me about Netapp.'
)

In [ ]:
search_results[0]

### Set up a LangChain RAG workflow to chat with the form

In [ ]:
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)


In [ ]:
retriever = neo4j_vector_store.as_retriever()

- Set up a RetrievalQAWithSourcesChain to carry out question answering
- You can check out the LangChain documentation for this chain [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.qa_with_sources.retrieval.RetrievalQAWithSourcesChain.html)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever
)

In [ ]:
def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain({"question": question},
        return_only_outputs=True,)
    print(textwrap.fill(response['answer'], 60))

- Ask a question!

In [ ]:
question = "What is Netapp's primary business?"

In [ ]:
prettychain(question)

In [ ]:
prettychain("Where is Netapp headquartered?")

In [ ]:
prettychain("""
    Tell me about Netapp. 
    Limit your answer to a single sentence.
""")

In [ ]:
prettychain("""
    Tell me about Apple. 
    Limit your answer to a single sentence.
""")

In [ ]:
prettychain("""
    Tell me about Apple. 
    Limit your answer to a single sentence.
    If you are unsure about the answer, say you don't know.
""")

### Ask you own question!
- Add your own question to the call to prettychain below to find out more about NetApp
- Here is NetApp's website if you want some inspiration: https://www.netapp.com/

In [ ]:
prettychain("""
    ADD YOUR OWN QUESTION HERE
""")